# <div style="background-color: #b74ff8ff; padding:10px; border-radius:10px; color: #ffffffff; font-weight: bold"> 📝 Ejercicio propuesto </div>
<div style="background-color: #abdbe3ff; padding:10px; border-radius:10px"> <p>Imagina que eres analista de datos en una empresa que recibe registros de clientes desde múltiples fuentes (formularios web, exportes legacy y hojas de cálculo). <em>Los datos llegan con errores típicos: marcadores de faltantes inconsistentes ("N/A", "NULL", "?", cadena vacía), símbolos en números ($, ,), fechas en formatos mixtos y texto con espacios/uso de mayúsculas irregular. Y debes organizar estos datos para presentarlos a los directivos. <strong>¿Cómo lo harías?</strong></em></p></div>

<div><p><strong>Para hacerlo te damos unas pistas 💡:</strong></p></div>
<div> 
    <ol style="list-style:none; padding-left:0;">
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#ef4444; color:#fff; font-weight:700; margin-right:10px;">1</span>
        Detectar valores nulos como <code>N/A</code>, <code>NULL</code>, <code>?</code> y convertirlos a <code>NaN</code>.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#f59e0b; color:#fff; font-weight:700; margin-right:10px;">2</span>
        Reemplazar marcadores faltantes por <code>NaN</code>.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#10b981; color:#fff; font-weight:700; margin-right:10px;">3</span>
        Convertir tipos de datos: números y fechas.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#60a5fa; color:#fff; font-weight:700; margin-right:10px;">4</span>
        Limpiar texto como <em>Customer Name</em> y <em>Gender</em>.
    </li>
    </ol>
</div>

>🚨Recuerda🚨 <em>Para realizar este ejercicio puedes encontrar toda la información teorica necesaria en <code>Notebook_demo</code> </em>


## 1) Dataset crudo (de ejemplo)
Generamos un DataFrame con problemas comunes para demostrar los pasos de limpieza.


In [1]:

import pandas as pd
import numpy as np
import os
import sqlite3



# Dataset de ejemplo con datos crudos y problemas comunes
df = pd.DataFrame({
    "Customer Name": ["John Doe ",      # Espacio al final
                      "  JANE DOE",     # Espacios al inicio y mayúsculas
                      "Alice smith",    # Inconsistente en minúsculas
                      "Mário   Santos", # Múltiples espacios + acento
                      "",               # Cadena vacía (faltante)
                      "NULL"],          # Texto "NULL" (faltante)
    "Income": ["$50,000",  # Símbolo + separador miles
               "N/A",      # Marcador faltante
               "$2,500",   # Símbolo + separador miles
               "10000",    # Numérico como texto
               "?",        # Marcador faltante
               "NULL"],    # Marcador faltante
    "Signup Date": ["12/05/2022",  # dd/mm/yyyy
                    "2021-07-30",  # yyyy-mm-dd
                    "15/07/2020",  # dd/mm/yyyy
                    "2020-01-05",  # yyyy-mm-dd
                    "N/A",         # faltante
                    "NULL"],       # faltante
    "Gender": ["M", "Female", "female", "F", "?", "NULL"]
})

print("Datos de ejemplo (crudos):")
df

# 💾 Guardar copia del dataset original para comparación posterior
df_original = df.copy()


Datos de ejemplo (crudos):



## 2) Detección de valores nulos
Antes de limpiar, verificamos cuántos **nulos reales** detecta pandas (notar que aún no convertimos marcadores como `N/A`, `NULL`, `?` a `NaN`).


In [63]:

print("Valores nulos antes de la limpieza:")
df.isna().sum()


Valores nulos antes de la limpieza:


Customer Name    0
Income           0
Signup Date      0
Gender           0
dtype: int64


## 3) Reemplazo de marcadores faltantes por `NaN`
Convertimos `""` (cadena vacía), `"?"`, `"N/A"` y `"NULL"` a `NaN` para que pandas los trate como faltantes.


In [64]:

df = df.replace(["", "?", "N/A", "NULL"], np.nan)
print("Valores nulos después del reemplazo:")
df.isna().sum()


Valores nulos después del reemplazo:


Customer Name    2
Income           3
Signup Date      2
Gender           2
dtype: int64


## 4) Conversión de tipos (número y fecha)
**Income:** remover símbolos (`$`, `,`) y convertir a numérico.  
**Signup Date:** convertir a `datetime`, tolerando formatos mixtos y errores.


In [65]:
# Income a numérico (con manejo seguro de NaN)
df['Income'] = df['Income'].astype(str).str.replace(r'[\$,]', '', regex=True)
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')

# Signup Date a datetime (dd/mm/yyyy y yyyy-mm-dd)
df['Signup Date'] = pd.to_datetime(df['Signup Date'], dayfirst=True, errors='coerce')

print("✅ Tipos de datos después de conversión:")
df.dtypes

✅ Tipos de datos después de conversión:


Customer Name            object
Income                  float64
Signup Date      datetime64[ns]
Gender                   object
dtype: object


## 5) Limpieza de texto (Customer Name y Gender)
- `Customer Name`: quitar espacios extra y normalizar a *Title Case*.  
- `Gender`: normalizar a categorías **Male/Female**.


In [66]:

# Customer Name: strip + espacios múltiples a uno + Title Case (manejo seguro)
df['Customer Name'] = df['Customer Name'].astype(str).str.strip()
df['Customer Name'] = df['Customer Name'].str.replace(r'\s+', ' ', regex=True)
df['Customer Name'] = df['Customer Name'].str.title()

# Gender: normalizar (manejo seguro de NaN)
df['Gender'] = df['Gender'].astype(str).str.strip().str.lower()
df['Gender'] = df['Gender'].replace({'m': 'male', 'male': 'male', 'f': 'female', 'female': 'female', 'nan': np.nan})
df['Gender'] = df['Gender'].str.capitalize()

print("✅ Limpieza de texto completada:")
df[['Customer Name','Gender']]


✅ Limpieza de texto completada:


,Customer Name,Gender
0,John Doe,Male
1,Jane Doe,Female
2,Alice Smith,Female
3,Mário Santos,Female
4,Nan,NaN
5,Nan,NaN



## 6) Reglas mínimas y resumen final
- (Opcional) Remover filas sin información clave.  
- Resetear el índice.  
- Revisar el estado final del DataFrame.


In [67]:

# Ejemplo: eliminar filas sin nombre e ingreso
df_clean = df.dropna(subset=['Customer Name', 'Income'], how='any').reset_index(drop=True)

print("✅ Filas válidas después de limpieza:")
print(f"   📊 Antes: {len(df)} filas")
print(f"   📊 Después: {len(df_clean)} filas")
print(f"   🗑️ Eliminadas: {len(df) - len(df_clean)} filas")

print("\n📋 Tipos de datos finales:")
display(df_clean.dtypes)

print("\n📄 DataFrame final:")
df_clean


✅ Filas válidas después de limpieza:
   📊 Antes: 6 filas
   📊 Después: 3 filas
   🗑️ Eliminadas: 3 filas

📋 Tipos de datos finales:


Customer Name            object
Income                  float64
Signup Date      datetime64[ns]
Gender                   object
dtype: object


📄 DataFrame final:


,Customer Name,Income,Signup Date,Gender
0,John Doe,50000.0,2022-05-12,Male
1,Alice Smith,2500.0,2020-07-15,Female
2,Mário Santos,10000.0,NaT,Female



## 7) Comparación **Antes vs. Después**
Para visualizar el impacto de la limpieza, mostramos una tabla comparativa con algunas filas del dataset original frente al dataset limpio.


In [68]:

print("📊 COMPARACIÓN COMPLETA - ANTES VS DESPUÉS")
print("=" * 60)

print("\n🔴 ANTES (datos crudos):")
print("-" * 30)
display(df_original)

print("\n🟢 DESPUÉS (datos limpios):")
print("-" * 30)
display(df_clean)

# Resumen de la transformación
print(f"\n📊 RESUMEN DE LA LIMPIEZA:")
print(f"   📊 Registros originales: {len(df_original)}")
print(f"   📊 Registros finales: {len(df_clean)}")
print(f"   🗑️ Registros eliminados: {len(df_original) - len(df_clean)}")
print(f"   📈 Tasa de conservación: {(len(df_clean)/len(df_original)*100):.1f}%")

📊 COMPARACIÓN COMPLETA - ANTES VS DESPUÉS

🔴 ANTES (datos crudos):
------------------------------


,Customer Name,Income,Signup Date,Gender
0,John Doe,"$50,000",12/05/2022,M
1,JANE DOE,N/A,2021-07-30,Female
2,Alice smith,"$2,500",15/07/2020,female
3,Mário Santos,10000,2020-01-05,F
4,,?,N/A,?
5,NULL,NULL,NULL,NULL



🟢 DESPUÉS (datos limpios):
------------------------------


,Customer Name,Income,Signup Date,Gender
0,John Doe,50000.0,2022-05-12,Male
1,Alice Smith,2500.0,2020-07-15,Female
2,Mário Santos,10000.0,NaT,Female



📊 RESUMEN DE LA LIMPIEZA:
   📊 Registros originales: 6
   📊 Registros finales: 3
   🗑️ Registros eliminados: 3
   📈 Tasa de conservación: 50.0%


In [70]:
# Crear carpeta "export" si no existe
os.makedirs("export", exist_ok=True)

print("Exportando dataset de clientes limpios...")

# 1) CSV - formato más universal para análisis de datos
df_clean.to_csv("export/clientes_limpios.csv", index=False, encoding="utf-8")
print("CSV exportado: export/clientes_limpios.csv")

# 2) Excel - formato empresarial estándar
df_clean.to_excel("export/clientes_limpios.xlsx", index=False)
print("Excel exportado: export/clientes_limpios.xlsx")

# 3) JSON - formato para APIs y aplicaciones web
df_clean.to_json("export/clientes_limpios.json", orient="records", indent=2, force_ascii=False)
print("JSON exportado: export/clientes_limpios.json")

# 4) SQL (SQLite) - base de datos para consultas complejas
con = sqlite3.connect("export/clientes_database.db")
df_clean.to_sql("clientes_limpios", con, if_exists="replace", index=False)
con.close()
print("Base de datos SQLite creada: export/clientes_database.db")



Exportando dataset de clientes limpios...
CSV exportado: export/clientes_limpios.csv
Excel exportado: export/clientes_limpios.xlsx
JSON exportado: export/clientes_limpios.json
Base de datos SQLite creada: export/clientes_database.db
